In [ ]:
import basic
import pandas as pd
import os
import matplotlib.pyplot as plt

import geopandas as gpd

import conda_scripts.plot_help as ph

In [ ]:
import flopy

In [ ]:
run = 'SWR_short_2lays_hiK'
info, swr_info, sfr_info, riv_keys_info = basic.load_params(run)

datestart = info['start_date']
numdays = info['numdays']
name = info['name']

basic.setup_folder(run)
out_folder = basic.out_folder(run)


print(datestart)
print(out_folder)


In [ ]:
ml = basic.load_model(load_fast = ['DIS', 'BAS6', 'UPW', 'NWT', 'WEL', 'GHB', 'OC', 'SFR', 'RCH', 'GAGE'])

In [ ]:
ml.gage.gage_data 

In [ ]:
import write_inflows

In [ ]:
write_inflows.run(model_name=run, m = ml, inflow_limit=20000)

In [ ]:
dat = ['rr__outflow_from_swr.gag',
 'rr__inflow_to_swr.gag',
 'rr__below_first_cell.gag',
 'dc__below_first_cell.gag',
 'rr__outflow.gag',
 'mark_west_creek__below_first_cell.gag',
 'mark_west_creek__inflow_to_rr.gag']

In [ ]:
import conda_scripts.arich_functions as af

In [ ]:
# f = pd.DataFrame(ml.gage.gage_data)
# f.loc[:,'name'] = dat
# f.loc[:,'rename'] = f.loc[:,'name'].str.strip('.gag').str.replace('rr', 'Russian River ').str.replace(
#                                                 'dc','dry creek ').str.replace(
#                                                 '_',' ').str.title()
# gages = pd.merge(pd.DataFrame(ml.sfr.reach_data), f ,
#          left_on = ['iseg', 'ireach'], right_on = ['gageloc','gagerch'] )
# # gages.loc[:,'name'] = dat

# shp = af.get_active_mod_df(ml.modelgrid, ml)

# gages = pd.merge(gages, shp, on = ['i','j'])

# gages= gpd.GeoDataFrame(gages, geometry = 'geometry', crs = 2226)
# gages = gages.set_geometry(gages.geometry.centroid)

In [ ]:
# fig, ax = basic.map_river(m = ml, add_basemap = True)

# gages.plot( ax = ax,marker = 'o', facecolor = 'w',edgecolor = 'k',zorder = 100)

# ph.label_points(ax, gages, colname = 'rename', fmt = "s")

In [ ]:
import Streamflows

In [ ]:
gages , flows = Streamflows.run(out_folder=out_folder, ml = ml, make_map= True)

In [ ]:
import plotly.express as px

In [ ]:
[flows[key].reset_index().insert(0,'gage',key) for key in flows]

def fixx(dfi, name):
    dfi.insert(0, 'Gage', name)
    
    return dfi

dfall = pd.concat([fixx(flows[key], key) for key in flows])

dfall.head()

In [ ]:
pd.to_datetime('Feb 7 2015') - pd.to_datetime('1/1/2014')

In [ ]:
# df = df.stack().to_frame('STAGE').reset_index()
# df = flows['rr__below_first_cell.gag']
# df = df.reset_index()
fig = px.line(dfall.reset_index(), x="Time", y="Flow", color = 'Gage')
fig.show()

In [ ]:
# def add_subindex_fields(file, h1, fields):
    

#     with open(file, 'r') as myfile:
#         if h1 in myfile.read():
#             present = True
#         else:
#             present = False
            
#     if not present:
#         with open(file, 'a') as myfile:
#             myfile.write('\n\n')
#             myfile.write("<h1>{:}</h1>\n".format(h1))

#             for k in fields:
#                 fileref = fields[k]
#                 part = """<a href="{:}">{:}<br></a>\n""".format(fileref, k)
#                 myfile.write(part)

# fields = {"Streamflow Gage map": r'streamflow\streamflow_gages_map.png',
# 'Russian River   Outflow From Swr':  r'streamflow\Russian River   Outflow From Swr.png',
# 'Russian River   Inflow To Swr':  r'streamflow\Russian River   Inflow To Swr.png',
# 'Russian River   Below First Cell':  r'streamflow\Russian River   Below First Cell.png',
# 'Dry Creek   Below First Cell':  r'streamflow\Dry Creek   Below First Cell.png',
# 'Russian River   Outflow':  r'streamflow\Russian River   Outflow.png',
# 'Mark West Creek  Below First Cell':  r'streamflow\Mark West Creek  Below First Cell.png',
# 'Mark West Creek  Inflow To Russian River ':  r'streamflow\Mark West Creek  Inflow To Russian River .png'}

basic.add_subindex_fields(os.path.join('versions', run, 'subindex.html'), 'sdfasdfasdf', fields )          

In [ ]:
def plot_streamflows(dat, model_ws, outfolder,  save = False):
    
    for filename in dat:
        # print(filename)
#         file = os.path.join(model_ws, 'Results', filename)
#         tab = pd.read_csv(file, 
#                           skiprows= [0],sep = '\s+')
#         f = "DATA: Time           Stage            Flow           Depth           Width      Midpt-Flow         Precip.              ET          Runoff     Conductance        HeadDiff       Hyd.Grad."
#         tab.columns = f.replace('DATA: ', '').split()

#         tab.index = pd.to_datetime('1/1/2014') + pd.to_timedelta(tab.Time, unit = 's')
#         tab = tab.drop(columns = 'Time')
#         tab = tab.loc[:,tab.abs().sum()>0]
#         axes = tab.plot(subplots = True, figsize = (8,6), fontsize = 12)

#         [{ax.text(1,.5, ax.get_legend_handles_labels()[1][0],
#                  transform=ax.transAxes, va = 'top'), ax.legend().remove()} for ax in axes]
        
        rename = filename.strip('.gag').replace('rr', 'Russian River ').replace(
                                                    'dc','dry creek ').replace(
                                                    '_',' ').title()
        plotname = os.path.join('streamflow', rename+'.png')
#         plt.suptitle(rename)
#         if save:
#             plotname = os.path.join(outfolder,'streamflow', rename+'.png')
#             print(f'saving {plotname}')
#             plt.savefig(plotname,dpi = 250, bbox_inches = 'tight')
        print("'{:}':  r'{:}',".format(rename, plotname))
        
dat = Streamflows.get_gage_names()
plot_streamflows(dat, ml.model_ws, out_folder, True)

In [ ]:
import plotly.express as px

In [ ]:
df = ISWRPQAQ.groupby(level = [0,5]).mean().loc[:,['STAGE']].unstack().droplevel(0,1).groupby(
pd.Grouper(level = 0, freq="1D")).mean().iloc[:,::5]
ax = df.plot(figsize = (10,10),cmap = 'jet')
ax.legend(loc = 'upper left', bbox_to_anchor = (1.05,.95))
ax.set_title('STAGE')
ax2 = ax.twinx()

df = df.stack().to_frame('STAGE').reset_index()

fig = px.line(df, x="TOTIME", y="STAGE", color='REACH')
fig.show()

In [ ]:
df.head()

In [ ]:

os.getcwd()

In [ ]:
fig.write_html('plotlytest.html')

In [ ]:
ax = ISWRPQAQ.groupby(level = [0,5]).mean().loc[:,['HEAD']].unstack().droplevel(0,1).groupby(
pd.Grouper(level = 0, freq="1W")).mean().iloc[:,::5].plot(figsize = (10,10),cmap = 'jet')
ax.legend(loc = 'upper left', bbox_to_anchor = (1.05,.95))
ax.set_title('GW Head')
ax2 = ax.twinx()

stg = ISWRPSTG.groupby(level = [0]).mean().groupby(
pd.Grouper(level = 0, freq="1W")).mean().iloc[:,115].plot(ax = ax2, c = 'k', lw = 4)

In [ ]:
ax = ISWRPRGF.groupby(level = [0,5]).sum().loc[:,['QAQFLOW']].unstack().droplevel(0,1).groupby(
pd.Grouper(level = 0, freq="1W")).mean().iloc[:,::5].plot(figsize = (10,10),cmap = 'jet')
ax.legend(loc = 'upper left', bbox_to_anchor = (1.05,.95))
ax2 = ax.twinx()
ax.set_title('GW Exchange')
stg = ISWRPSTG.groupby(level = [0]).mean().groupby(
pd.Grouper(level = 0, freq="1W")).mean().iloc[:,115].plot(ax = ax2, c = 'k', lw = 4)

In [ ]:
df.head()

In [ ]:
df = ISWRPRGF.groupby(level = [0,5]).sum().loc[:,['QAQFLOW']].unstack().droplevel(0,1).groupby(
pd.Grouper(level = 0, freq="1W")).mean().iloc[:,::5]
df = df.stack().to_frame('Q').reset_index()

fig = px.line(df, x="TOTTIME", y="Q", color='RCHGRP',title='GW Exchange')
fig.write_html('GW_exchange.html')
fig.show()

In [ ]:
ISWRPSTR

In [ ]:


df = ISWRPSTR.query("REACH==116").iloc[:,::5]
df = df.reset_index()

fig = px.line(df, x="TOTIME", y="STRFLOW", title='Structure Flow')
# fig.write_html('GW_exchange.html')
fig.show()

In [ ]:
import postprocess

In [ ]:
ISWRPQAQ, ISWRPRGF, ISWRPSTG, ISWRPSTR, ISWRPQM = postprocess.run(run,riv_only = True,max_reach = 76)

In [ ]:
import Hydrographs

In [ ]:
.01*86400

In [ ]:
basic.plot_aquifer_props_pilot_points(ml, 'pilot_points', out_folder, load_best = False)

In [ ]:
Hydrographs.run(run, ml = ml, reload = True, add_temp=False)

In [ ]:
basic.plot_maps(ml, run)

In [ ]:
import SFRtoSWR

In [ ]:
import basic
import write_inflows

In [ ]:
ml = basic.load_model()

In [ ]:
write_inflows.run(run, m = ml, inflow_limit=10000)

In [ ]:
import post_process_heads
import numpy as np
import cartopy.crs as ccrs

In [ ]:
post_process_heads.run(run_name = run,head_frequency = 3, ml = ml, add_basemap=True)

In [ ]:
hds, hdsobj = basic.get_heads(ml)

In [ ]:
hds.shape

In [ ]:
hds, other= basic.get_heads(ml)

In [ ]:
hds

In [ ]:
hds

In [ ]:
props, pval = basic.load_pilot_point_calibrated_props(ml, 'pilot_points', load_best= False)

In [ ]:
pval

In [ ]:
np.unique(ml.upw.hk.array[0].reshape((-1)))

In [ ]:
fig, ax = plt.subplots(
    figsize = (5,5), subplot_kw=dict(projection=ccrs.epsg(2226)))

mapview = flopy.plot.PlotMapView(ml, ax = ax)
# arr = np.ma.array(ml.modelgrid.saturated_thick(hds[0]), mask = ml.bas6.ibound.array[0] == 0)

arr = ml.modelgrid.saturated_thick(hds)*np.stack([props['hk1'], props['hk2'], props['hk3']])
arr = np.ma.array(arr.sum(axis=0), mask = ml.bas6.ibound.array[0] == 0)


ac = mapview.plot_array(arr)
basic.set_bounds(ax, 'mirabel')
ax.set_title('Transmissivity')
plt.colorbar(ac, shrink = .5)

In [ ]:
ff = props['hk1']

ff[ff<1e5]

In [ ]:
plt.imshow(arr)
plt.colorbar()

In [ ]:
1000/86400

In [ ]:
sy = np.genfromtxt("RR_2022\pilot_points\sy1.txt")
f = sy

print(np.isnan(f).all())
f[f>1e5] = np.nan

f = f.reshape(-1)

f = f[~np.isnan(f)]
print(np.mean(f))
ax = pd.Series(f).hist()
# # ax.axvline(1000)
# ax.set_xlim(xmax = 1001)

In [ ]:
hk = np.genfromtxt("RR_2022\pilot_points\hk2.txt")
f = hk

print(np.isnan(f).all())
f[f>1e5] = np.nan

f = f.reshape(-1)

f = f[~np.isnan(f)]
print(np.median(f))
print(1000/(np.median(f)*86400))
ax = pd.Series(f*86400).hist()
# # ax.axvline(1000)
# ax.set_xlim(xmax = 1001)

In [ ]:
hk = np.genfromtxt("RR_2022\pilot_points\hk1.txt")
f = hk

print(np.isnan(f).all())
f[f>1e5] = np.nan

f = f.reshape(-1)

f = f[~np.isnan(f)]
print(np.median(f))
print(1000/(np.median(f)*86400))
ax = pd.Series(f*86400).hist()
# # ax.axvline(1000)
# ax.set_xlim(xmax = 1001)

In [ ]:
f = props['hk1']

print(np.isnan(f).all())
f[f>1e5] = np.nan

f = f.reshape(-1)

f = f[~np.isnan(f)]

f

In [ ]:
fig, axes = plt.subplots(1,2,
    figsize = (10,5), subplot_kw=dict(projection=ccrs.epsg(2226)))

axes = axes.flat

for lay in range(2):
    ax = axes[lay]
    mapview = flopy.plot.PlotMapView(ml, ax = ax)
    # arr = np.ma.array(ml.modelgrid.saturated_thick(hds[0]), mask = ml.bas6.ibound.array[0] == 0)
    
    k = np.stack([props['hk1'], props['hk2'], props['hk3']])
    arr = ml.modelgrid.saturated_thick(hds)*k
    arr = np.ma.array(arr[lay], mask = ml.bas6.ibound.array[lay] == 0)

    
    ac = mapview.plot_array(arr, vmax = 50)
    basic.set_bounds(ax, 'mirabel')
    
ax.set_title('Transmissivity')
plt.colorbar(ac, shrink = .2)

In [ ]:
grid = gpd.read_file("GIS\grid.shp")

In [ ]:
hk.reshape(-1

In [ ]:


fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 1, 1)
row = 159
xsect = flopy.plot.PlotCrossSection(model=ml, line={"Row": row})
patches = xsect.plot_ibound()
linecollection = xsect.plot_grid()
t = ax.set_title(f"Row {row} Cross-Section with IBOUND Boundary Conditions")
ax.set_xlim(xmin=5000, xmax = 14000)
ax.set_ylim(ymax = 200)

In [ ]:
cbbobj = os.path.join(ml.model_ws, 'Results', )
times = cbbobj.get_times()
qx = cbbobj.get_data(text="flow right face", totim=times[-1])[0]
qy = np.zeros((nlay, nrow, ncol), dtype=float)
qz = cbbobj.get_data(text="flow lower face", totim=times[-1])[0]



In [ ]:

flopy.utils.postprocessing.get_water_table(hds, -999.)

In [ ]:
fig, ax = plt.subplots( figsize = (5,5), subplot_kw=dict(projection=ccrs.epsg(2226)))

mapview = flopy.plot.PlotMapView(ml, ax = ax)
# arr = np.ma.array(ml.modelgrid.saturated_thick(hds[0]), mask = ml.bas6.ibound.array[0] == 0)
arr = np.ma.array(flopy.utils.postprocessing.get_water_table(hds, -999.), mask = ml.bas6.ibound.array[0] == 0)


ac = mapview.plot_array(arr,vmin = 0)
mapview.contour_array(arr, levels = np.arange(0, 100, 20), colors = 'k')
basic.set_bounds(ax, 'mirabel')
ax.set_title('Water Table Elevation')
plt.colorbar(ac, shrink = .5)

In [ ]:
import contextily as ctx

In [ ]:
swr_reaches = gpd.read_file("GIS/SWR_Reaches.shp")
sfr_reaches = gpd.read_file("SFR_files/only_sfr_cells.shp")

In [ ]:
def plot_all_heads(hds, swr_reaches, sfr_reaches, ml, title):
        fig, axes = plt.subplots(1, 3, figsize=(20, 8.5), subplot_kw=dict(projection=ccrs.epsg(2226)))
        axes = axes.flat

        for i, hdslayer in enumerate(hds):
            mapview = flopy.plot.PlotMapView(ml,ax = axes[i])
            axes[i].set_facecolor('lightgrey')
            # linecollection = mapview.plot_grid(linewidth = .3)
            # ax.set_title(f"layer {lay+1} elevation (feet)")
            # Hide X and Y axes tick marks
            axes[i].set_xticks([])
            axes[i].set_yticks([])
            # hdslayer[hdslayer==-999] = np.nan
            # im = axes[i].imshow(hdslayer, vmin=0, vmax=75)
            axes[i].set_title("Layer {}".format(i + 1))
            ma = np.ma.array(hdslayer,mask = m.bas6.ibound.array[i]==0)
            quadmesh = mapview.plot_array(ma, vmax = 50,vmin  =10,cmap = 'gist_ncar_r')
            # ctr = axes[i].contour(ma, colors="k", linewidths=0.5, vmin = 0, vmax = 75)

            set_bounds(axes[i], 'mirabel')
            if add_basemap:
                ctx.add_basemap(axes[i], crs = 2226)

            swr_reaches.plot(ax = axes[i], facecolor = "None", edgecolor = 'k')
            sfr_reaches.plot(ax = axes[i], facecolor = "None", edgecolor = 'grey')

        # fig.delaxes(axes[-1])
        fig.subplots_adjust(right=0.8)
        cbar_ax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
        fig.colorbar(quadmesh, cax=cbar_ax, label="Head");
        
plot_all_heads(hds, swr_reaches, sfr_reaches, ml, title)

In [ ]:
well = gpd.read_file('GIS\wells.shp')

In [ ]:
import conda_scripts.plot_help as ph

In [ ]:
bounds

In [ ]:
mpl.cm.get_cmap()

In [ ]:
import matplotlib as mpl
cmap=mpl.cm.jet
bounds=np.arange(0,100,10)

fig, ax = plt.subplots()
# cmap.set_over('0.25')
# cmap.set_under('0.75')
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb2 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                norm=norm,
                                boundaries= bounds,
                                extend='both',
                                ticks=bounds,
                                spacing='proportional',
                                orientation='horizontal')

In [ ]:
def plot_heads(hds, swr_reaches, sfr_reaches, ml, title):
    fig, axes = plt.subplots(2,1, figsize = (5,8), gridspec_kw = {'hspace':.001},
                             subplot_kw=dict(projection=ccrs.epsg(2226)))

    ax = axes[0]
    mapview = flopy.plot.PlotMapView(ml, ax = ax)
    # arr = np.ma.array(ml.modelgrid.saturated_thick(hds[0]), mask = ml.bas6.ibound.array[0] == 0)
    sat = ml.modelgrid.saturated_thick(hds)
    sat = sat[0] + sat[1]
    arr = np.ma.array(sat, mask = ml.bas6.ibound.array[0] == 0)

    bounds=np.arange(0,100,10)
    norm = mpl.colors.BoundaryNorm(bounds, 256)
    
    # ac = mapview.plot_array(arr, vmin=0, vmax=90, cmap='gist_ncar_r')
    ac = mapview.plot_array(arr, norm = norm, cmap = 'gist_ncar_r')
    mapview.contour_array(arr, levels = np.arange(0, 100, 20), colors = 'k')
    basic.set_bounds(ax, 'mirabel')
    ax.set_title('Saturated Thickness')
    plt.colorbar(ac, shrink = .5,ax = ax, label = 'feet')

    ctx.add_basemap(ax,
                    crs=2226,
                    source=r"C:\GIS\basemap\SRP_hydro.tif")

    wt = flopy.utils.postprocessing.get_water_table(hds,nodata = -999)
    wt = np.ma.array(wt, mask = ml.bas6.ibound.array[0] == 0)

    mapview = flopy.plot.PlotMapView(ml, ax = axes[1])
    bounds=np.arange(0,80,5)
    norm = mpl.colors.BoundaryNorm(bounds, 256)
    ac = mapview.plot_array(wt, norm = norm, cmap = 'rainbow_r')
    mapview.contour_array(wt, levels = np.arange(0, 100, 20), colors = 'k')
    basic.set_bounds(axes[1], 'mirabel')
    axes[1].set_title('Watertable Elevation')
    plt.colorbar(ac, shrink = .5,ax = axes[1], label = 'feet')
    ctx.add_basemap(axes[1],
                    crs=2226,
                    source=r"C:\GIS\basemap\SRP_hydro.tif")

    [(swr_reaches.plot(ax = axi), sfr_reaches.plot(ax = axi), well.plot(ax = axi, facecolor = 'None')) for axi in axes]
    
    fig.suptitle(title)
    
plot_heads(hds, swr_reaches, sfr_reaches, ml, 'asdf')

In [ ]:
fig, ax = plt.subplots( figsize = (8,5), subplot_kw=dict(projection=ccrs.epsg(2226)))

mapview = flopy.plot.PlotMapView(ml, ax = ax)
# arr = np.ma.array(ml.modelgrid.saturated_thick(hds[0][0]), mask = ml.bas6.ibound.array[0] == 0)
sat = ml.modelgrid.saturated_thick(hds[0])
sat = sat[0] + sat[1]
arr = np.ma.array(sat, mask = ml.bas6.ibound.array[0] == 0)


ac = mapview.plot_array(arr, vmin = 0, vmax = 100, cmap = 'BrBG')
mapview.contour_array(arr, levels = np.arange(0, 100, 20), colors = 'k')
basic.set_bounds(ax, 'mirabel')
ax.set_title('Saturated Thickness')
plt.colorbar(ac, shrink = .5)

ctx.add_basemap(ax,
                crs=2226,
                source=r"C:\GIS\basemap\SRP_hydro.tif")

In [ ]:
ml.bas6.export("GIS\ibound_update_2023.shp")

In [ ]:
flopy.utils.postprocessing.get_water_table(hds, nodata = [-999])

In [ ]:
flopy.utils.postprocessing.get_transmissivities(hds[0],ml, sctop = 0)

In [ ]:
fig, ax = plt.subplots( figsize = (5,5), subplot_kw=dict(projection=ccrs.epsg(2226)))

mapview = flopy.plot.PlotMapView(ml, ax = ax)
arr = np.ma.array(flopy.utils.postprocessing.get_transmissivities(hds[0],ml).sum(axis=0), mask = ml.bas6.ibound.array[0] == 0)

basic.set_bounds(ax, 'mirabel')
ac = mapview.plot_array(arr)
plt.colorbar(ac)

In [ ]:
ml.dis.check()

In [ ]:
ml = basic.load_model()